# Testinnsending av personlig skattemelding med næringspesifikasjon

Denne demoen er ment for å vise hvordan flyten for et sluttbrukersystem kan hente et utkast, gjøre endringer og validere/kontrollere det mot Skatteetatens apier, for å sende det inn via Altinn3.

In [ ]:
try:
    from altinn3 import *
    from skatteetaten_api import get_access_token, base64_decode_response, decode_dokument, valider
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from pathlib import Path
except ImportError as e:
    print("Mangler en eller avhengighet, installer dem via pip, se requierments.txt fil for detaljer")
    raise ImportError(e)



## Variabler som må endres i test
Før testscriptet kjøres må testparten som brukes være på allowlist med tilgang til skattemeldingen.

In [ ]:
# oppdater med skattemelding-api-relay fra aktuelt testmiljø
skatteetaten_url = "https://idporten-api-sbstest.sits.no"
altinn3_applikasjon = "skd/formueinntekt-skattemelding-v2"

# oppdater med test fødselsnummerene du har fått tildelt
fnr = "09845297174"
typeInnsending = "komplett"
inntektsår = "2022"

## Generer ID-porten token
Tokenet er gyldig i 86400 sekunder, rekjørt denne biten om du ikke har nådd Altinn3 biten innen 300 sekunder

# Hent utkast og gjeldende
Her legger vi inn fødselsnummeret vi logget oss inn med. Dersom du velger et annet fødselsnummer så må den du logget på med ha tilgang til skattemeldingen du ønsker å hente

Parten nedenfor er brukt for internt test, pass på bruk deres egne testparter når dere tester


In [ ]:
idporten_header = get_access_token()
s = requests.Session()
s.headers = dict(idporten_header)

### Utkast

In [ ]:
url_utkast = f'{skatteetaten_url}/api/skattemelding/v2/utkast/{inntektsår}/{fnr}'
r_utkast = s.get(url_utkast)
r_utkast


### Gjeldende

In [ ]:
url_gjeldende = f'{skatteetaten_url}/api/skattemelding/v2/{inntektsår}/{fnr}'
r_gjeldende = s.get(url_gjeldende)
r_gjeldende 

#### Fastsatt
Her får en _http 404_ om vedkommende ikke har noen fastsetting, rekjørt denne etter du har sendt inn og fått tilbakemdling i Altinn at den har blitt behandlet, du skal nå ha en fastsatt skattemelding om den har blitt sent inn som Komplett

In [ ]:
url_fastsatt = f'{skatteetaten_url}/api/skattemelding/v2/fastsatt/{inntektsår}/{fnr}'
r_fastsatt = s.get(url_fastsatt)
r_fastsatt

## Svar fra hent gjeldende 

### Gjeldende dokument referanse: 
I responsen på alle api kallene, være seg utkast/fastsatt eller gjeldene, så følger det med en dokumentreferanse. 
For å kalle valider tjenesten, er en avhengig av å bruke korrekt referanse til gjeldende skattemelding. 

Cellen nedenfor henter ut gjeldende dokumentreferanse printer ut responsen fra hent gjeldende kallet 

In [ ]:
sjekk_svar = r_gjeldende

sme_og_naering_respons = xmltodict.parse(sjekk_svar.text)
skattemelding_base64 = sme_og_naering_respons['skattemeldingOgNaeringsspesifikasjonforespoerselResponse']['dokumenter']['skattemeldingdokument']
sme_base64 = skattemelding_base64["content"]
dokref = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]['skattemeldingdokument']['id']
decoded_sme_xml = decode_dokument(skattemelding_base64)
sme_utkast = xml.dom.minidom.parseString(decoded_sme_xml["content"]).toprettyxml()

print(f"Responsen fra hent gjeldende ser slik ut, gjeldende dokumentrerefanse er {dokref}\n")
print(xml.dom.minidom.parseString(sjekk_svar.text).toprettyxml())

sme_dict = xmltodict.parse(decoded_sme_xml['content'])
partsnummer = sme_dict['skattemelding']['partsreferanse']

In [ ]:
with open(f"../../../src/resources/eksempler/{inntektsår}/Naeringspesifikasjon-enk-v3.xml", 'r') as f:
    naering_enk_xml = f.read()

innsendingstype = typeInnsending
naeringsspesifikasjoner_enk_b64 = base64.b64encode(naering_enk_xml.encode("utf-8"))
naeringsspesifikasjoner_enk_b64 = str(naeringsspesifikasjoner_enk_b64.decode("utf-8"))
skattemeldingPersonligSkattepliktig_base64=sme_base64 #bruker utkastet uten noen endringer
naeringsspesifikasjoner_base64=naeringsspesifikasjoner_enk_b64
dok_ref=dokref

valider_konvlutt_v2 = """
<?xml version="1.0" encoding="utf-8" ?>
<skattemeldingOgNaeringsspesifikasjonRequest xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingognaeringsspesifikasjon:request:v2">
    <dokumenter>
        <dokument>
            <type>skattemeldingPersonlig</type>
            <encoding>utf-8</encoding>
            <content>{sme_base64}</content>
        </dokument>
        <dokument>
            <type>naeringsspesifikasjon</type>
            <encoding>utf-8</encoding>
            <content>{naeringsspeifikasjon_base64}</content>
        </dokument>
    </dokumenter>
    <dokumentreferanseTilGjeldendeDokument>
        <dokumenttype>skattemeldingPersonlig</dokumenttype>
        <dokumentidentifikator>{dok_ref}</dokumentidentifikator>
    </dokumentreferanseTilGjeldendeDokument>
    <inntektsaar>{inntektsår}</inntektsaar>
    <innsendingsinformasjon>
        <innsendingstype>{innsendingstype}</innsendingstype>
        <opprettetAv>TurboSkatt</opprettetAv>
    </innsendingsinformasjon>
</skattemeldingOgNaeringsspesifikasjonRequest>
""".replace("\n","")


naering_enk = valider_konvlutt_v2.format(
    sme_base64=skattemeldingPersonligSkattepliktig_base64,
    naeringsspeifikasjon_base64=naeringsspesifikasjoner_base64,
    dok_ref=dok_ref,
    inntektsår=inntektsår,
    innsendingstype=innsendingstype)

# Valider utkast sme med næringsopplysninger

In [ ]:
valider_respons = valider(naering_enk, inntektsår, s, fnr, idporten_header, )
resultatAvValidering = xmltodict.parse(valider_respons.text)["skattemeldingOgNaeringsspesifikasjonResponse"]["resultatAvValidering"]

if valider_respons:
    print(resultatAvValidering + "\n")
    print(xml.dom.minidom.parseString(valider_respons.text).toprettyxml())
else:
    print(valider_respons.status_code, valider_respons.headers, valider_respons.text)

# Altinn 3

1. Hent Altinn Token
2. Oppretter en ny instans av skjemaet
3. Last opp vedlegg til skattemeldingen
4. Oppdater skattemelding xml med referanse til vedlegg_id fra altinn3.
5. Laster opp skattemeldingen og næringsopplysninger som et vedlegg

In [ ]:
#1
altinn_header = hent_altinn_token(idporten_header)
instans_data = opprett_ny_instans_med_inntektsaar(altinn_header, inntektsaar=inntektsår, appnavn=altinn3_applikasjon, fnr=fnr, testmiljoe=None)
instans_data

In [ ]:
#Last opp skattemeldingen
# noinspection PyNoneFunctionAssignment
req_send_inn = last_opp_skattedata(instans_data, altinn_header,
                                   xml=naering_enk,
                                   data_type="skattemeldingOgNaeringsspesifikasjon",
                                   appnavn=altinn3_applikasjon)
req_send_inn

### Sett status til bekreftelse

In [ ]:
req_bekreftelse = endre_prosess_status(instans_data, altinn_header, "next", appnavn=altinn3_applikasjon)
req_bekreftelse

### Sett statusen klar til henting av skatteetaten. 

In [ ]:
req_bekreftelse = endre_prosess_status(instans_data, altinn_header, "next", appnavn=altinn3_applikasjon)
req_bekreftelse

### Sjekk status på altinn3 instansen om skatteetaten har hentet instansen.
Denne statusen vil til å begynne med ha verdien "none". Oppdatering skjer når skatteetaten har behandlet innsendingen.
- Ved **komplett**-innsending vil status oppdateres til Godkjent/Avvist når innsendingen er behandlet.
- Ved **ikkeKomplett**-innsending vil status oppdateres til Tilgjengelig når innsendingen er behandlet. Etter innsending via SME vil den oppdateres til Godkjent/Avvist etter behandling.

In [ ]:
instans_etter_bekreftelse = hent_instans(instans_data, altinn_header, appnavn=altinn3_applikasjon)
response_data = instans_etter_bekreftelse.json()
print(f"Instans-status: {response_data['status']['substatus']}")

### Hent/vis instans

In [ ]:
instans_json = hent_instans(instans_data, altinn_header, appnavn=altinn3_applikasjon)
instans_json.json()

### Se innsending i Altinn

Ta en slurk av kaffen og klapp deg selv på ryggen, du har nå sendt inn, la byråkratiet gjøre sin ting... og det tar litt tid. Pt så sjekker skatteeaten hos Altinn3 hvert 30 sek om det har kommet noen nye innsendinger. Skulle det gå mer enn et par minutter så har det mest sansynlig feilet. 

Før dere feilmelder noe til skatteetaten så må dere minimum ha med enten en korrelasjons-id eller instans-id for at vi skal kunne feilsøke

# Ikke komplett skattemelding
1. Når du har fått svar i altinn innboksen, så kan du gå til 
 https://skatt-sbstest.sits.no/web/skattemeldingen/2022
2. Her vil du se næringsinntekter overført fra skattemeldingen
3. Når du har sendt inn i SME så vil du kunne se i altinn instansen at den har blitt avsluttet
4. Kjør cellen nedenfor for å se at du har fått en ny fastsatt skattemelding og næringsopplysninger


In [ ]:
print("Resultat av hent fastsatt før fastsetting")
print(r_fastsatt.text)
print("Resultat av hent fastsatt etter fastsetting")

r_fastsatt2 = s.get(url_fastsatt)
r_fastsatt2.text
#r_fastsatt.elapsed.total_seconds()